# Google Ads API Test - OAuth & Search Terms Reporting

This notebook demonstrates how to:
1. Set up OAuth authentication with Google Ads API
2. Test the connection
3. Run search terms reporting

## Prerequisites
- Google Ads API enabled in Google Cloud Console
- OAuth client credentials (desktop application)
- Developer token approved by Google
- Customer ID for your Google Ads account


In [18]:
# Cell 1: Environment Setup
import os
import json

# Set your credentials
os.environ['GOOGLE_ADS_DEVELOPER_TOKEN'] = 'YOUR_DEVELOPER_TOKEN_HERE'
os.environ['GOOGLE_ADS_CUSTOMER_ID'] = 'YOUR_CUSTOMER_ID_HERE'

print("🔧 Environment Setup")
print("=" * 30)
print(f"✅ Developer Token: {os.environ['GOOGLE_ADS_DEVELOPER_TOKEN'][:10]}...")
print(f"✅ Customer ID: {os.environ['GOOGLE_ADS_CUSTOMER_ID']}")

# Check for OAuth client file
oauth_file = "client_secret.json"
if os.path.exists(oauth_file):
    print(f"✅ Found: {oauth_file}")
else:
    print(f"❌ Missing: {oauth_file}")


🔧 Environment Setup
✅ Developer Token: LjyGdgTk0U...
✅ Customer ID: YOUR_CUSTOMER_ID_HERE
✅ Found: client_secret.json


In [19]:
# Cell 2: OAuth Authentication
print("🔄 OAuth Authentication")
print("=" * 25)

try:
    from google_auth_oauthlib.flow import InstalledAppFlow
    
    # OAuth configuration (including Google Sheets API)
    SCOPES = [
        "https://www.googleapis.com/auth/adwords",
        "https://www.googleapis.com/auth/spreadsheets"
    ]
    client_secret_file = "client_secret.json"
    
    print("🚀 Starting OAuth flow...")
    print("This will open a browser window for authentication.")
    
    # Create and run the OAuth flow
    flow = InstalledAppFlow.from_client_secrets_file(
        client_secret_file,
        scopes=SCOPES
    )
    
    # Run the flow - opens browser automatically
    credentials = flow.run_local_server(port=0, access_type="offline", prompt="consent")
    
    print("✅ OAuth authentication successful!")
    print(f"Access token: {credentials.token[:20]}...")
    print(f"Refresh token: {credentials.refresh_token[:20]}...")
    
    # Store credentials for next cells
    refresh_token = credentials.refresh_token
    
    # Read client config for client_id and client_secret
    with open(client_secret_file, 'r') as f:
        client_config = json.load(f)
    
    client_id = client_config['installed']['client_id']
    client_secret = client_config['installed']['client_secret']
    
    print(f"\n💾 Save this refresh token for future use:")
    print(f"{refresh_token}")
    
except Exception as e:
    print(f"❌ OAuth authentication failed: {e}")
    import traceback
    traceback.print_exc()


🔄 OAuth Authentication
🚀 Starting OAuth flow...
This will open a browser window for authentication.
Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=YOUR_CLIENT_ID_HERE&redirect_uri=http%3A%2F%2Flocalhost%3A49368%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fadwords+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fspreadsheets&state=1LwnUALerVZ5usCKZ2sIY1k5xDEPsK&access_type=offline&prompt=consent
✅ OAuth authentication successful!
Access token: ya29.a0AQQ_BDRwtWLOD...
Refresh token: [REFRESH_TOKEN_SAVED_LOCALLY]

💾 Save this refresh token for future use:
[REFRESH_TOKEN_SAVED_LOCALLY]


In [20]:
# Cell 3: Test Google Ads API Connection
print("🔗 Testing Google Ads API Connection")
print("=" * 40)

try:
    from google.ads.googleads.client import GoogleAdsClient
    
    # Configure client with OAuth credentials
    config = {
        'developer_token': os.getenv('GOOGLE_ADS_DEVELOPER_TOKEN'),
        'client_id': client_id,
        'client_secret': client_secret,
        'refresh_token': refresh_token,
        'use_proto_plus': True,
    }
    
    print("🔄 Finding working API version and service...")
    
    # Try different API versions and service names
    working_version = None
    working_service = None
    client = None
    ga_service = None
    
    # First try without specifying version (use default)
    print("  Trying default version (no version specified)...")
    try:
        client = GoogleAdsClient.load_from_dict(config)
        print(f"    ✅ Client created with default version: {client.version}")
        
        # Try different service names
        service_names = ['GoogleAdsService', 'GoogleAdsServiceClient', 'GoogleAds', 'google_ads_service']
        
        for service_name in service_names:
            try:
                ga_service = client.get_service(service_name)
                print(f"    ✅ Service {service_name} works with default version!")
                working_version = client.version
                working_service = service_name
                break
            except Exception as e:
                print(f"    ❌ Service {service_name} failed: {str(e)[:50]}...")
                continue
                
    except Exception as e:
        print(f"    ❌ Default version failed: {str(e)[:50]}...")
    
    # If default didn't work, try different API versions (prioritize supported versions)
    if not working_version:
        print("  📋 Note: v21 is the current supported version (as of 2024)")
        print("  📋 v18 and older versions are deprecated and may be blocked")
        print("  📋 Using GitHub installation with proper service modules")
        for version in ['v21', 'v20', 'v19', 'v17', 'v16', 'v15', 'v14', 'v13', 'v12']:
            print(f"  Trying API version {version}...")
            try:
                client = GoogleAdsClient.load_from_dict(config, version=version)
                print(f"    ✅ Client created for {version}")
                
                # Try different service names
                service_names = ['GoogleAdsService', 'GoogleAdsServiceClient', 'GoogleAds', 'google_ads_service']
                
                for service_name in service_names:
                    try:
                        ga_service = client.get_service(service_name)
                        print(f"    ✅ Service {service_name} works with {version}!")
                        working_version = version
                        working_service = service_name
                        break
                    except Exception as e:
                        print(f"    ❌ Service {service_name} failed: {str(e)[:50]}...")
                        continue
                
                if working_version:
                    break
                    
            except Exception as e:
                print(f"    ❌ API version {version} failed: {str(e)[:50]}...")
                continue
    
    # If still no working version, try v18 as last resort (deprecated but might work)
    if not working_version:
        print("  Trying deprecated v18 as last resort...")
        try:
            client = GoogleAdsClient.load_from_dict(config, version='v18')
            print(f"    ✅ Client created for v18 (DEPRECATED)")
            
            ga_service = client.get_service('GoogleAdsService')
            print(f"    ⚠️  Service GoogleAdsService works with v18 (DEPRECATED)")
            working_version = 'v18'
            working_service = 'GoogleAdsService'
            
        except Exception as e:
            print(f"    ❌ API version v18 also failed: {str(e)[:50]}...")
    
    if working_version and working_service:
        print(f"\n🎉 SUCCESS! Using API version {working_version} with service {working_service}")
        
        # Warning for deprecated version
        if working_version == 'v18':
            print("⚠️  WARNING: Using deprecated API version v18")
            print("   This version may be blocked by Google. Consider upgrading to a supported version.")
        
        # Test with a simple customer query
        customer_id = os.getenv('GOOGLE_ADS_CUSTOMER_ID')
        query = f'SELECT customer.id, customer.descriptive_name FROM customer WHERE customer.id = {customer_id}'
        
        print(f"🔍 Testing query on customer {customer_id}...")
        
        try:
            response = ga_service.search(customer_id=customer_id, query=query)
            
            for row in response:
                print(f"✅ SUCCESS! Connected to Google Ads API!")
                print(f"  Customer ID: {row.customer.id}")
                print(f"  Account Name: {row.customer.descriptive_name}")
                break
            else:
                print("⚠️  Connection works but no customer data found")
                print("This might be due to:")
                print("- Customer ID access permissions")
                print("- Account restrictions")
                print("- API limitations")
                
        except Exception as e:
            if 'deprecated' in str(e).lower() or 'unsupported_version' in str(e).lower():
                print(f"❌ API version {working_version} is blocked by Google")
                print("   The version is deprecated and requests are being rejected.")
                print("   We need to find a supported version.")
                working_version = None  # Reset to try other versions
            else:
                raise e
    else:
        print("\n❌ No working API version found")
        print("This might be due to:")
        print("- Google Ads API library version issues")
        print("- Missing API version modules")
        print("- Library installation problems")
        print("- OAuth credentials issues")
        
        print("\n🔧 Try these solutions:")
        print("1. The library may be missing API version modules. Try:")
        print("   conda run -n CAB_V1_20250504 pip uninstall google-ads -y")
        print("   conda run -n CAB_V1_20250504 pip install git+https://github.com/googleads/google-ads-python.git")
        print("2. Check your OAuth credentials are valid")
        print("3. Verify your developer token is approved")
        print("4. Make sure Google Ads API is enabled in Google Cloud Console")
        print("5. The GitHub installation should have proper service modules")
        
except Exception as e:
    print(f"❌ Connection test failed: {e}")
    import traceback
    traceback.print_exc()


🔗 Testing Google Ads API Connection
🔄 Finding working API version and service...
  Trying default version (no version specified)...
    ✅ Client created with default version: None
    ✅ Service GoogleAdsService works with default version!
  📋 Note: v21 is the current supported version (as of 2024)
  📋 v18 and older versions are deprecated and may be blocked
  📋 Using GitHub installation with proper service modules
  Trying API version v21...
    ✅ Client created for v21
    ✅ Service GoogleAdsService works with v21!

🎉 SUCCESS! Using API version v21 with service GoogleAdsService
🔍 Testing query on customer YOUR_CUSTOMER_ID_HERE...
✅ SUCCESS! Connected to Google Ads API!
  Customer ID: YOUR_CUSTOMER_ID_HERE
  Account Name: Wafeq MCC


In [15]:
# Cell 4: Search Terms Reporting
print("🔍 Search Terms Reporting")
print("=" * 25)

try:
    # Use the working client from previous cell
    if 'client' in locals() and 'ga_service' in locals() and 'working_version' in locals():
        customer_id = str(os.getenv('GOOGLE_ADS_CUSTOMER_ID'))
        
        # Comprehensive query for search terms with all details (v21 compatible)
        query = """
        SELECT
            search_term_view.search_term,
            search_term_view.status,
            campaign.name,
            campaign.id,
            ad_group.name,
            ad_group.id,
            metrics.impressions,
            metrics.clicks,
            metrics.cost_micros,
            metrics.ctr,
            metrics.average_cpc,
            metrics.conversions,
            metrics.conversions_value,
            metrics.cost_per_conversion,
            segments.date
        FROM search_term_view
        WHERE segments.date DURING LAST_7_DAYS
        ORDER BY metrics.impressions DESC
        LIMIT 50
        """
        
        print(f"🔍 Getting search terms for customer {customer_id}...")
        
        try:
            response = ga_service.search(customer_id=customer_id, query=query)
        except Exception as e:
            if 'manager account' in str(e).lower() or 'requested_metrics_for_manager' in str(e).lower():
                print("⚠️  This is a Manager Account (MCC). Getting client accounts first...")
                
                # Get list of accessible customer accounts
                accessible_customers_query = """
                SELECT customer_client.id, customer_client.descriptive_name, customer_client.manager
                FROM customer_client
                WHERE customer_client.status = 'ENABLED'
                """
                
                print("🔍 Getting accessible client accounts...")
                accessible_response = ga_service.search(customer_id=customer_id, query=accessible_customers_query)
                
                client_accounts = []
                for row in accessible_response:
                    client_id = row.customer_client.id
                    client_name = row.customer_client.descriptive_name
                    is_manager = row.customer_client.manager
                    
                    if not is_manager:  # Only get actual client accounts, not sub-managers
                        client_accounts.append({
                            'id': client_id,
                            'name': client_name
                        })
                
                if client_accounts:
                    print(f"✅ Found {len(client_accounts)} client accounts:")
                    for client in client_accounts:
                        print(f"  - {client['id']}: {client['name']}")
                    
                    # Try to get search terms from the first client account
                    first_client = client_accounts[0]
                    print(f"\\n🔍 Getting search terms from client account {first_client['id']} ({first_client['name']})...")
                    
                    # Convert customer ID to string for v21 compatibility
                    client_customer_id = str(first_client['id'])
                    response = ga_service.search(customer_id=client_customer_id, query=query)
                else:
                    print("❌ No client accounts found under this manager account")
                    raise Exception("No accessible client accounts found")
                    
            elif 'search_term_view' in str(e).lower():
                print("⚠️  search_term_view not available, trying alternative query...")
                # Alternative query using keyword_view
                query = """
                SELECT
                    keyword_view.resource_name,
                    keyword_view.criterion_id,
                    metrics.impressions,
                    metrics.clicks,
                    metrics.cost_micros,
                    metrics.ctr,
                    metrics.average_cpc
                FROM keyword_view
                WHERE segments.date DURING LAST_7_DAYS
                ORDER BY metrics.impressions DESC
                LIMIT 20
                """
                response = ga_service.search(customer_id=str(customer_id), query=query)
            else:
                raise e
        
        search_terms = []
        for row in response:
            # Handle both search_term_view and keyword_view
            if hasattr(row, 'search_term_view'):
                search_term = row.search_term_view.search_term
                status = row.search_term_view.status.name if hasattr(row.search_term_view.status, 'name') else str(row.search_term_view.status)
            elif hasattr(row, 'keyword_view'):
                search_term = f"Keyword ID: {row.keyword_view.criterion_id}"
                status = "KEYWORD"
            else:
                search_term = "Unknown"
                status = "UNKNOWN"
            
            # Get campaign and ad group info
            campaign_name = row.campaign.name if hasattr(row, 'campaign') else "N/A"
            campaign_id = row.campaign.id if hasattr(row, 'campaign') else "N/A"
            ad_group_name = row.ad_group.name if hasattr(row, 'ad_group') else "N/A"
            ad_group_id = row.ad_group.id if hasattr(row, 'ad_group') else "N/A"
            
            # Get metrics
            impressions = row.metrics.impressions
            clicks = row.metrics.clicks
            cost = row.metrics.cost_micros / 1_000_000  # Convert from micros to currency
            ctr = row.metrics.ctr
            avg_cpc = row.metrics.average_cpc / 1_000_000  # Convert from micros to currency
            conversions = row.metrics.conversions if hasattr(row.metrics, 'conversions') else 0
            conversions_value = row.metrics.conversions_value / 1_000_000 if hasattr(row.metrics, 'conversions_value') else 0  # Convert from micros
            cost_per_conversion = row.metrics.cost_per_conversion / 1_000_000 if hasattr(row.metrics, 'cost_per_conversion') else 0  # Convert from micros
            
            # Get date
            date = row.segments.date if hasattr(row, 'segments') and hasattr(row.segments, 'date') else "N/A"
            
            search_terms.append({
                'search_term': search_term,
                'status': status,
                'campaign_name': campaign_name,
                'campaign_id': campaign_id,
                'ad_group_name': ad_group_name,
                'ad_group_id': ad_group_id,
                'date': date,
                'impressions': impressions,
                'clicks': clicks,
                'cost': cost,
                'ctr': ctr,
                'avg_cpc': avg_cpc,
                'conversions': conversions,
                'conversions_value': conversions_value,
                'cost_per_conversion': cost_per_conversion
            })
        
        if search_terms:
            print(f"✅ Found {len(search_terms)} search terms!")
            print("\n📊 Comprehensive Search Terms Report (Last 7 Days):")
            print("=" * 120)
            
            # Show detailed results
            for i, term in enumerate(search_terms[:10], 1):
                print(f"\n🔍 #{i} - {term['search_term']}")
                print(f"   📅 Date: {term['date']}")
                print(f"   🎯 Campaign: {term['campaign_name']} (ID: {term['campaign_id']})")
                print(f"   📦 Ad Group: {term['ad_group_name']} (ID: {term['ad_group_id']})")
                print(f"   📊 Status: {term['status']}")
                print(f"   👁️  Impressions: {term['impressions']:,}")
                print(f"   🖱️  Clicks: {term['clicks']:,}")
                print(f"   📈 CTR: {term['ctr']:.2%}")
                print(f"   💰 Cost: ${term['cost']:.2f}")
                print(f"   💵 Avg CPC: ${term['avg_cpc']:.2f}")
                print(f"   🎯 Conversions: {term['conversions']:.1f}")
                print(f"   💎 Conversion Value: ${term['conversions_value']:.2f}")
                print(f"   📊 Cost/Conversion: ${term['cost_per_conversion']:.2f}")
                print("-" * 80)
            
            # Summary statistics
            total_impressions = sum(term['impressions'] for term in search_terms)
            total_clicks = sum(term['clicks'] for term in search_terms)
            total_cost = sum(term['cost'] for term in search_terms)
            total_conversions = sum(term['conversions'] for term in search_terms)
            total_conversion_value = sum(term['conversions_value'] for term in search_terms)
            overall_ctr = (total_clicks / total_impressions) if total_impressions > 0 else 0
            overall_cpa = (total_cost / total_conversions) if total_conversions > 0 else 0
            
            print("\n📈 SUMMARY STATISTICS:")
            print("=" * 50)
            print(f"📊 Total Search Terms: {len(search_terms)}")
            print(f"👁️  Total Impressions: {total_impressions:,}")
            print(f"🖱️  Total Clicks: {total_clicks:,}")
            print(f"📈 Overall CTR: {overall_ctr:.2%}")
            print(f"💰 Total Cost: ${total_cost:.2f}")
            print(f"🎯 Total Conversions: {total_conversions:.1f}")
            print(f"💎 Total Conversion Value: ${total_conversion_value:.2f}")
            print(f"📊 Overall CPA: ${overall_cpa:.2f}")
            
            # Show status breakdown
            status_counts = {}
            for term in search_terms:
                status = term['status']
                status_counts[status] = status_counts.get(status, 0) + 1
            
            print(f"\n📋 STATUS BREAKDOWN:")
            for status, count in status_counts.items():
                print(f"   {status}: {count} search terms")
            
        else:
            print("ℹ️  No search terms found for the last 7 days")
            print("This could mean:")
            print("- No campaigns are running")
            print("- No search terms have been recorded")
            print("- Date range has no data")
            print("- Insufficient permissions")
    else:
        print("❌ No working connection found. Run the previous cells first.")
        
except Exception as e:
    print(f"❌ Search terms query failed: {e}")
    print("This might be due to:")
    print("- No campaigns with search terms")
    print("- Insufficient permissions")
    print("- API limitations")
    import traceback
    traceback.print_exc()


🔍 Search Terms Reporting
🔍 Getting search terms for customer YOUR_CUSTOMER_ID_HERE...


Request made: ClientCustomerId: YOUR_CUSTOMER_ID_HERE, Host: googleads.googleapis.com, Method: /google.ads.googleads.v21.services.GoogleAdsService/Search, RequestId: GL5n16rd6Y9a3NdOlULLgA, IsFault: True, FaultMessage: Metrics cannot be requested for a manager account. To retrieve metrics, issue separate requests against each client account under the manager account.


⚠️  This is a Manager Account (MCC). Getting client accounts first...
🔍 Getting accessible client accounts...
✅ Found 1 client accounts:
  - 1092369762: Wafeq Ads Account
\n🔍 Getting search terms from client account 1092369762 (Wafeq Ads Account)...
✅ Found 50 search terms!

📊 Comprehensive Search Terms Report (Last 7 Days):

🔍 #1 - دفترة
   📅 Date: 2025-09-22
   🎯 Campaign: WFQ_GA_SEM_AO_0_Comp_Web_All_KSA_AR_PRO_Conv_16052025 (ID: 22558125330)
   📦 Ad Group: Daftra (ID: 179272681829)
   📊 Status: ADDED
   👁️  Impressions: 1,069
   🖱️  Clicks: 7
   📈 CTR: 0.65%
   💰 Cost: $61.09
   💵 Avg CPC: $8.73
   🎯 Conversions: 2.0
   💎 Conversion Value: $0.00
   📊 Cost/Conversion: $30.55
--------------------------------------------------------------------------------

🔍 #2 - دفترة
   📅 Date: 2025-09-21
   🎯 Campaign: WFQ_GA_SEM_AO_0_Comp_Web_All_KSA_AR_PRO_Conv_16052025 (ID: 22558125330)
   📦 Ad Group: Daftra (ID: 179272681829)
   📊 Status: ADDED
   👁️  Impressions: 1,066
   🖱️  Clicks: 5
   📈 C

In [ ]:
# Cell 5: Campaign Performance Overview
print("📊 Campaign Performance Overview")
print("=" * 35)

try:
    if 'client' in locals() and 'ga_service' in locals() and 'working_version' in locals():
        customer_id = os.getenv('GOOGLE_ADS_CUSTOMER_ID')
        
        # Query for campaign performance
        query = """
        SELECT
            campaign.id,
            campaign.name,
            campaign.status,
            metrics.impressions,
            metrics.clicks,
            metrics.cost_micros,
            metrics.ctr,
            metrics.average_cpc
        FROM campaign
        WHERE segments.date DURING LAST_7_DAYS
        ORDER BY metrics.impressions DESC
        LIMIT 10
        """
        
        print(f"🔍 Getting campaign performance for customer {customer_id}...")
        response = ga_service.search(customer_id=customer_id, query=query)
        
        campaigns = []
        for row in response:
            campaign_id = row.campaign.id
            campaign_name = row.campaign.name
            status = row.campaign.status.name
            impressions = row.metrics.impressions
            clicks = row.metrics.clicks
            cost = row.metrics.cost_micros / 1_000_000
            ctr = row.metrics.ctr
            avg_cpc = row.metrics.average_cpc / 1_000_000
            
            campaigns.append({
                'id': campaign_id,
                'name': campaign_name,
                'status': status,
                'impressions': impressions,
                'clicks': clicks,
                'cost': cost,
                'ctr': ctr,
                'avg_cpc': avg_cpc
            })
        
        if campaigns:
            print(f"✅ Found {len(campaigns)} campaigns!")
            print("\n📊 Campaign Performance (Last 7 Days):")
            print("-" * 100)
            print(f"{'Campaign Name':<25} {'Status':<10} {'Impressions':<12} {'Clicks':<8} {'CTR':<8} {'Cost':<10}")
            print("-" * 100)
            
            for campaign in campaigns:
                name = campaign['name'][:24] if len(campaign['name']) > 24 else campaign['name']
                print(f"{name:<25} {campaign['status']:<10} {campaign['impressions']:<12,} {campaign['clicks']:<8,} {campaign['ctr']:<8.2%} ${campaign['cost']:<9.2f}")
        else:
            print("ℹ️  No campaign data found for the last 7 days")
    else:
        print("❌ No working connection found. Run the previous cells first.")
        
except Exception as e:
    print(f"❌ Campaign query failed: {e}")
    import traceback
    traceback.print_exc()


📊 Campaign Performance Overview
❌ No working connection found. Run the previous cells first.


In [ ]:
# Cell 6: Export to Google Sheets
print("📊 Exporting Search Terms to Google Sheets")
print("=" * 45)

try:
    # Check if we have search terms data from previous cell
    if 'search_terms' in locals() and search_terms:
        print(f"📋 Found {len(search_terms)} search terms to export")
        
        # Install required packages if not already installed
        try:
            import gspread
            from google.auth import default
            print("✅ gspread library available")
        except ImportError:
            print("📦 Installing gspread library...")
            import subprocess
            import sys
            subprocess.check_call([sys.executable, "-m", "pip", "install", "gspread", "google-auth"])
            import gspread
            from google.auth import default
            print("✅ gspread library installed")
        
        # Authenticate with Google Sheets
        print("🔐 Authenticating with Google Sheets...")
        
        # Use the same OAuth credentials from the Google Ads API
        if 'credentials' in locals():
            # Check if credentials have the required scope
            if hasattr(credentials, 'scopes') and 'https://www.googleapis.com/auth/spreadsheets' in credentials.scopes:
                # Authorize gspread with existing credentials
                gc = gspread.authorize(credentials)
                print("✅ Google Sheets authentication successful")
            else:
                print("⚠️  OAuth credentials don't include Google Sheets scope")
                print("💡 You need to re-run the OAuth authentication cell (Cell 2) to get the updated scopes")
                print("   Or use the CSV export method (Cell 7) instead")
                raise Exception("Insufficient OAuth scopes for Google Sheets API")
            
            # Create or open a spreadsheet
            from datetime import datetime
            spreadsheet_name = f"Google Ads Search Terms - {customer_id} - {datetime.now().strftime('%Y-%m-%d')}"
            
            try:
                # Try to open existing spreadsheet
                spreadsheet = gc.open(spreadsheet_name)
                print(f"📄 Opened existing spreadsheet: {spreadsheet_name}")
            except gspread.SpreadsheetNotFound:
                # Create new spreadsheet
                spreadsheet = gc.create(spreadsheet_name)
                print(f"📄 Created new spreadsheet: {spreadsheet_name}")
            
            # Get the first worksheet
            worksheet = spreadsheet.sheet1
            
            # Clear existing data
            worksheet.clear()
            
            # Prepare headers
            headers = [
                'Search Term', 'Status', 'Campaign Name', 'Campaign ID', 
                'Ad Group Name', 'Ad Group ID', 'Date', 'Impressions', 
                'Clicks', 'CTR', 'Cost', 'Avg CPC', 'Conversions', 
                'Conversion Value', 'Cost Per Conversion'
            ]
            
            # Add headers
            worksheet.append_row(headers)
            print("📝 Added headers to spreadsheet")
            
            # Add data rows
            for term in search_terms:
                row = [
                    term['search_term'],
                    term['status'],
                    term['campaign_name'],
                    term['campaign_id'],
                    term['ad_group_name'],
                    term['ad_group_id'],
                    str(term['date']),
                    term['impressions'],
                    term['clicks'],
                    f"{term['ctr']:.2%}",
                    f"${term['cost']:.2f}",
                    f"${term['avg_cpc']:.2f}",
                    f"{term['conversions']:.1f}",
                    f"${term['conversions_value']:.2f}",
                    f"${term['cost_per_conversion']:.2f}"
                ]
                worksheet.append_row(row)
            
            print(f"✅ Successfully exported {len(search_terms)} search terms to Google Sheets")
            print(f"🔗 Spreadsheet URL: {spreadsheet.url}")
            print(f"📊 Spreadsheet Name: {spreadsheet_name}")
            
            # Add summary data
            summary_row = ['', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
            worksheet.append_row(summary_row)
            
            # Add summary statistics
            total_impressions = sum(term['impressions'] for term in search_terms)
            total_clicks = sum(term['clicks'] for term in search_terms)
            total_cost = sum(term['cost'] for term in search_terms)
            total_conversions = sum(term['conversions'] for term in search_terms)
            total_conversion_value = sum(term['conversions_value'] for term in search_terms)
            overall_ctr = (total_clicks / total_impressions) if total_impressions > 0 else 0
            overall_cpa = (total_cost / total_conversions) if total_conversions > 0 else 0
            
            summary_headers = ['SUMMARY', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
            worksheet.append_row(summary_headers)
            
            summary_data = [
                f'Total Search Terms: {len(search_terms)}',
                f'Total Impressions: {total_impressions:,}',
                f'Total Clicks: {total_clicks:,}',
                f'Overall CTR: {overall_ctr:.2%}',
                f'Total Cost: ${total_cost:.2f}',
                f'Total Conversions: {total_conversions:.1f}',
                f'Total Conversion Value: ${total_conversion_value:.2f}',
                f'Overall CPA: ${overall_cpa:.2f}'
            ]
            
            for i, summary_item in enumerate(summary_data):
                row = [''] * 15
                row[i] = summary_item
                worksheet.append_row(row)
            
            print("📈 Added summary statistics to spreadsheet")
            
        else:
            print("❌ No OAuth credentials found. Please run the OAuth authentication cell first.")
            
    else:
        print("❌ No search terms data found. Please run the search terms reporting cell first.")
        
except Exception as e:
    print(f"❌ Google Sheets export failed: {e}")
    print("💡 Make sure you have:")
    print("   1. Run the OAuth authentication cell first")
    print("   2. Run the search terms reporting cell first")
    print("   3. Google Sheets API enabled in your Google Cloud project")
    import traceback
    traceback.print_exc()


📊 Exporting Search Terms to Google Sheets
📋 Found 50 search terms to export
✅ gspread library available
🔐 Authenticating with Google Sheets...
✅ Google Sheets authentication successful
❌ Google Sheets export failed: APIError: [403]: Request had insufficient authentication scopes.
💡 Make sure you have:
   1. Run the OAuth authentication cell first
   2. Run the search terms reporting cell first
   3. Google Sheets API enabled in your Google Cloud project


Traceback (most recent call last):
  File "/var/folders/cc/bgt_fv_50sq8sz8sf8d9grg40000gn/T/ipykernel_26943/3848146546.py", line 52, in <module>
    spreadsheet = gc.open(spreadsheet_name)
                  ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/envs/CAB_V1_20250504/lib/python3.12/site-packages/gspread/client.py", line 145, in open
    spreadsheet_files, response = self._list_spreadsheet_files(title, folder_id)
                                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/envs/CAB_V1_20250504/lib/python3.12/site-packages/gspread/client.py", line 118, in _list_spreadsheet_files
    response = self.http_client.request("get", url, params=params)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/envs/CAB_V1_20250504/lib/python3.12/site-packages/gspread/http_client.py", line 128, in request
    raise APIError(response)
gspread.exceptions.APIError: APIError: [403]: Request had insufficient authenticat

In [17]:
# Cell 7: Export to CSV (Alternative Method)
print("📊 Exporting Search Terms to CSV")
print("=" * 35)

try:
    # Check if we have search terms data from previous cell
    if 'search_terms' in locals() and search_terms:
        print(f"📋 Found {len(search_terms)} search terms to export")
        
        import csv
        from datetime import datetime
        
        # Create filename with timestamp
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        filename = f"google_ads_search_terms_{customer_id}_{timestamp}.csv"
        
        # Write to CSV
        with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
            fieldnames = [
                'search_term', 'status', 'campaign_name', 'campaign_id', 
                'ad_group_name', 'ad_group_id', 'date', 'impressions', 
                'clicks', 'ctr', 'cost', 'avg_cpc', 'conversions', 
                'conversions_value', 'cost_per_conversion'
            ]
            
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            
            for term in search_terms:
                writer.writerow({
                    'search_term': term['search_term'],
                    'status': term['status'],
                    'campaign_name': term['campaign_name'],
                    'campaign_id': term['campaign_id'],
                    'ad_group_name': term['ad_group_name'],
                    'ad_group_id': term['ad_group_id'],
                    'date': str(term['date']),
                    'impressions': term['impressions'],
                    'clicks': term['clicks'],
                    'ctr': f"{term['ctr']:.4f}",
                    'cost': f"{term['cost']:.2f}",
                    'avg_cpc': f"{term['avg_cpc']:.2f}",
                    'conversions': f"{term['conversions']:.1f}",
                    'conversions_value': f"{term['conversions_value']:.2f}",
                    'cost_per_conversion': f"{term['cost_per_conversion']:.2f}"
                })
        
        print(f"✅ Successfully exported {len(search_terms)} search terms to CSV")
        print(f"📄 File saved as: {filename}")
        print(f"📁 Full path: {os.path.abspath(filename)}")
        
        # Show summary
        total_impressions = sum(term['impressions'] for term in search_terms)
        total_clicks = sum(term['clicks'] for term in search_terms)
        total_cost = sum(term['cost'] for term in search_terms)
        total_conversions = sum(term['conversions'] for term in search_terms)
        
        print(f"\\n📈 Export Summary:")
        print(f"   📊 Total Search Terms: {len(search_terms)}")
        print(f"   👁️  Total Impressions: {total_impressions:,}")
        print(f"   🖱️  Total Clicks: {total_clicks:,}")
        print(f"   💰 Total Cost: ${total_cost:.2f}")
        print(f"   🎯 Total Conversions: {total_conversions:.1f}")
        
        print(f"\\n💡 You can now:")
        print(f"   1. Open the CSV file in Excel or Google Sheets")
        print(f"   2. Upload it to Google Sheets manually")
        print(f"   3. Import it into any data analysis tool")
        
    else:
        print("❌ No search terms data found. Please run the search terms reporting cell first.")
        
except Exception as e:
    print(f"❌ CSV export failed: {e}")
    import traceback
    traceback.print_exc()


📊 Exporting Search Terms to CSV
📋 Found 50 search terms to export
✅ Successfully exported 50 search terms to CSV
📄 File saved as: google_ads_search_terms_YOUR_CUSTOMER_ID_HERE_20250924_232422.csv
📁 Full path: /Users/rakesh/Documents/MCP_Adwords/google_ads_search_terms_YOUR_CUSTOMER_ID_HERE_20250924_232422.csv
\n📈 Export Summary:
   📊 Total Search Terms: 50
   👁️  Total Impressions: 10,726
   🖱️  Total Clicks: 170
   💰 Total Cost: $2167.20
   🎯 Total Conversions: 8.1
\n💡 You can now:
   1. Open the CSV file in Excel or Google Sheets
   2. Upload it to Google Sheets manually
   3. Import it into any data analysis tool


## 🎉 Success!

If you've reached this point, you have successfully:

1. ✅ **Set up OAuth authentication** with Google Ads API
2. ✅ **Connected to Google Ads API** using your credentials
3. ✅ **Retrieved search terms data** from your campaigns
4. ✅ **Viewed campaign performance** overview

## 🔧 Next Steps

You can now:
- Modify the queries to get different data
- Change date ranges (e.g., `LAST_30_DAYS`, `THIS_MONTH`)
- Add more metrics to your reports
- Create automated reporting scripts
- Build an MCP server for Google Ads API

## 📚 Useful Resources

- [Google Ads API Documentation](https://developers.google.com/google-ads/api/docs)
- [Query Builder](https://developers.google.com/google-ads/api/fields/overview)
- [Python Client Library](https://developers.google.com/google-ads/api/docs/client-libs/python)
- [OAuth Setup Guide](https://developers.google.com/google-ads/api/docs/client-libs/python/oauth-web)
